In [1]:
# import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains, Chrome
import time

In [2]:
df_all=pd.DataFrame()
#code=['E445167','E445186','E446807']


In [3]:
item_list=pd.read_excel('item list.xlsx')

In [4]:
item_list.shape

(100, 11)

In [16]:
URL_List=[]
for item in item_list['item code'][58:61]:
    #get subpage data
    if item_list[item_list['item code']==item]['url'].values[0] not in URL_List:
        URL = item_list[item_list['item code']==item]['url'].values[0]
    else:
        URL = item_list[item_list['item code']==item]['url'].values[1]
    URL_List.append(URL)
    print(URL)
    driver = webdriver.Chrome()

    driver.get(URL) #replacing the requests.get step

    #get the code list of available unit trusts
    subhtml = driver.page_source #replacing the html.text

    soup = BeautifulSoup(subhtml, "html.parser")
    soup.prettify()
    
    #Click all もっと見る
    click=0
    try:
        while ('もっと見る' == soup.find_all(class_="uq-ec-show-more uq-ec-button-reset uq-ec-button--large uq-ec-cursor-pointer uq-ec-mb-spacing-06")[0].text)and(click<=200):
            click+=1
            python_button = driver.find_elements_by_xpath("//*[@id='root']/div/section/div[3]/section/div/div[2]/div[2]/button")[0]
            python_button.click()

            subhtml = driver.page_source #replacing the html.text
            soup = BeautifulSoup(subhtml, "html.parser")
            soup.prettify()
    except:
        pass
    
    dict_df={}
    count=0
    for i in soup.find_all(class_="uq-ec-caption uq-ec-caption--standard-color-primary-dark uq-ec-text-align-left uq-ec-review__user-info uq-ec-review__grey-text uq-ec-caption--standard uq-ec-text-transform-normal"):
        if ('性' in i.text)or ('回答しない'in i.text) or ('その他'in i.text):
            if count != 0:    
                while len(dict_df['{}_{}'.format(item,count)]) < 5:
                    dict_df['{}_{}'.format(item,count)].append('nan')
                    
            count+=1
            dict_df['{}_{}'.format(item,count)]=[]
            dict_df['{}_{}'.format(item,count)].append(i.text)
        elif ('代' in i.text)|('歳' in i.text)|('ヶ月'in i.text):
            dict_df['{}_{}'.format(item,count)].append(i.text)
        elif '身長' in i.text:
            while len(dict_df['{}_{}'.format(item,count)]) <2:
                dict_df['{}_{}'.format(item,count)].append('nan')
            dict_df['{}_{}'.format(item,count)].append(i.text)
        elif '体重' in i.text:
            while len(dict_df['{}_{}'.format(item,count)]) <3:
                dict_df['{}_{}'.format(item,count)].append('nan')
            dict_df['{}_{}'.format(item,count)].append(i.text)
        elif '足のサイズ' in i.text:
            while len(dict_df['{}_{}'.format(item,count)]) <4:
                dict_df['{}_{}'.format(item,count)].append('nan')
            dict_df['{}_{}'.format(item,count)].append(i.text)


    while len(dict_df['{}_{}'.format(item,count)]) < 5:
        dict_df['{}_{}'.format(item,count)].append('nan')
    
    df=pd.DataFrame(dict_df)
    
    # Size and Rating
    dict2={'Size':[],'Rating':[],'Item_Code':[]}
    for i in soup.find_all(class_="uq-ec-caption uq-ec-caption--standard-color-primary-dark uq-ec-text-align-left uq-ec-review__purchase--information uq-ec-caption--standard uq-ec-text-transform-normal"):
        dict2['Size'].append(i.text)
        dict2['Item_Code'].append(item)
    
    for i in soup.find_all(class_="uq-ec-rating-static uq-ec-rating"):
          dict2['Rating'].append(str(i).count("full"))
    df2=pd.DataFrame(dict2)
    
    df1=df.transpose()
    df1.reset_index(inplace=True)
    
    df_new=pd.concat([df1,df2], axis=1)
    
    df_all=pd.concat([df_all,df_new],axis=0)
    driver.close()



https://www.uniqlo.com/jp/ja/products/E444598-000/00/reviews


C:\Users\jacksuen\AppData\Local\Temp/ipykernel_15784/4086949560.py:25: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  python_button = driver.find_elements_by_xpath("//*[@id='root']/div/section/div[3]/section/div/div[2]/div[2]/button")[0]


https://www.uniqlo.com/jp/ja/products/E442533-000/00/reviews


C:\Users\jacksuen\AppData\Local\Temp/ipykernel_15784/4086949560.py:25: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  python_button = driver.find_elements_by_xpath("//*[@id='root']/div/section/div[3]/section/div/div[2]/div[2]/button")[0]


https://www.uniqlo.com/jp/ja/products/E422370-000/00/reviews


C:\Users\jacksuen\AppData\Local\Temp/ipykernel_15784/4086949560.py:25: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  python_button = driver.find_elements_by_xpath("//*[@id='root']/div/section/div[3]/section/div/div[2]/div[2]/button")[0]


In [ ]:
dict_df

In [ ]:
df_all.columns=['id','gender','AgeGroup','Height','Weight','Foot_Size','Product_Size','Rating','Item_Code']

In [18]:
df_all

,index,0,1,2,3,4,Size,Rating,Item_Code
0,445334_1,女性,50代,身長: 156 - 160cm,体重: 46 - 50kg,nan,購入サイズ: L,4,445334
1,445334_2,女性,20代,身長: 161 - 165cm,体重: 51 - 55kg,足のサイズ: 25.5cm,購入サイズ: M,4,445334
2,445334_3,女性,nan,nan,nan,nan,購入サイズ: S,1,445334
3,445334_4,女性,30代,身長: 151 - 155cm,体重: 41 - 45kg,足のサイズ: 22.0cm,購入サイズ: S,5,445334
4,445334_5,女性,nan,nan,nan,nan,購入サイズ: L,5,445334
...,...,...,...,...,...,...,...,...,...
1005,444593_1006,男性,50代,身長: 166 - 170cm,体重: 66 - 70kg,足のサイズ: 25.5cm,購入サイズ: 85,5,444593
1006,444593_1007,男性,40代,身長: 176 - 180cm,体重: 86 - 90kg,足のサイズ: 27.5cm,購入サイズ: 91,5,444593
1007,444593_1008,男性,10代,身長: 166 - 170cm,体重: 41 - 45kg,足のサイズ: 26.0cm,購入サイズ: 79,5,444593
1008,444593_1009,男性,30代,身長: 166 - 170cm,体重: 61 - 65kg,足のサイズ: 26.5cm,購入サイズ: 76,4,444593


In [11]:
df_all['Item_Code'].unique()

array([445334, 446484], dtype=int64)

In [20]:
df_all[df_all['Item_Code']==446919]

,index,0,1,2,3,4,Size,Rating,Item_Code
0,446919_1,男性,nan,身長: 176 - 180cm,nan,足のサイズ: 26.0cm,購入サイズ: M,5,446919
1,446919_2,女性,50代,身長: 151 - 155cm,体重: 56 - 60kg,足のサイズ: 22.5cm,購入サイズ: XS,5,446919
2,446919_3,男性,15 - 19歳,身長: 166 - 170cm,体重: 71 - 75kg,足のサイズ: 26.5cm,購入サイズ: XL,5,446919
3,446919_4,男性,60代以上,身長: 171 - 175cm,体重: 71 - 75kg,足のサイズ: 26.0cm,購入サイズ: L,5,446919
4,446919_5,男性,20代,身長: 166 - 170cm,体重: 56 - 60kg,足のサイズ: 26.0cm,購入サイズ: M,5,446919
...,...,...,...,...,...,...,...,...,...
31,446919_32,男性,60代以上,身長: 181 - 185cm,体重: 81 - 85kg,足のサイズ: 26.5cm,購入サイズ: XL,5,446919
32,446919_33,男性,60代以上,身長: 176 - 180cm,体重: 71 - 75kg,足のサイズ: 26.5cm,購入サイズ: XL,5,446919
33,446919_34,男性,30代,身長: 171 - 175cm,体重: 51 - 55kg,足のサイズ: 26.0cm,購入サイズ: XL,5,446919
34,446919_35,男性,40代,身長: 166 - 170cm,体重: 71 - 75kg,足のサイズ: 27.5cm,購入サイズ: XL,5,446919


In [ ]:
length_dict = {key: len(value) for key, value in dict_df.items()}
length_dict

In [19]:
df_all.to_csv('Sample_445334_444593.csv', encoding='utf-8', index=False)